# Исследование базы данных StackOverflow

## Описание проекта

В самостоятельном проекте вы будете работать с базой данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.


## Данные


Вы будете работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах вы найдёте информацию и о более поздних оценках, которые эти посты получили. 

## Цель

С помощью SQL посчитать и визуализировать ключевые метрики сервис-системы вопросов и ответов о программировании.

### Задание 1
Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
'''
   SELECT COUNT(parent_id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.post_types AS pt ON p.post_type_id = pt.id
WHERE type = 'Question'
  AND (score > 300 
   OR favorites_count >= 100)

'''

### Задание 2
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.



In [ ]:
'''
WITH
type_q AS
(SELECT id
FROM stackoverflow.post_types
WHERE type = 'Question'),
count_q AS
(SELECT CAST(creation_date AS DATE) AS date_q,
       COUNT(p.id) AS count_q
FROM stackoverflow.posts AS p
JOIN type_q AS q ON p.post_type_id = q.id
WHERE CAST(creation_date AS DATE) BETWEEN '2008-11-01' AND '2008-11-18'
GROUP BY date_q)

SELECT ROUND(AVG(count_q.count_q), 0)
FROM count_q
    
'''

### Задание 3
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.


In [ ]:
'''
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users AS u
JOIN stackoverflow.badges AS b ON u.id = b.user_id
WHERE CAST(b.creation_date AS DATE) = CAST(u.creation_date AS DATE)
'''

### Задание 4

Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?



In [ ]:
'''
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id = u.id
JOIN stackoverflow.votes AS v ON p.id = v.post_id 
WHERE display_name = 'Joel Coehoorn' 
'''

### Задание 5
Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
'''
  SELECT *,
       ROW_NUMBER() OVER (ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id 
'''

### Задание 6

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
'''
  SELECT u.id AS user_id,
       COUNT(v.id) AS votes 
FROM stackoverflow.users AS u
JOIN stackoverflow.votes AS v ON u.id = v.user_id
JOIN stackoverflow.vote_types AS vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY u.id
ORDER BY votes DESC, user_id DESC
LIMIT 10
'''

### Задание 7

Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
'''
  SELECT u.id,
       COUNT(b.id) AS count_badges,
       DENSE_RANK() OVER (ORDER BY COUNT(b.id) DESC) AS rn
FROM stackoverflow.users AS u
JOIN stackoverflow.badges AS b ON u.id = b.user_id
WHERE CAST(b.creation_date AS DATE) BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY u.id
ORDER BY count_badges DESC, u.id ASC
LIMIT 10     
'''

### Задание 8

Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
'''
 SELECT p.title,
       u.id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY u.id))
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id = u.id
WHERE p.title != '' 
  AND score != 0
'''

### Задание 9
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
'''
WITH
users_budges AS 
(SELECT b.user_id,
        COUNT(b.id) AS count_badges
FROM  stackoverflow.badges AS b 
GROUP BY b.user_id
HAVING COUNT(b.id) > 1000)

SELECT p.title
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id = u.id
JOIN users_budges AS ub ON u.id = ub.user_id
WHERE p.title != ''
'''

### Задание 10

Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:

- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.


In [ ]:
'''
SELECT id,
       views,
       CASE
          WHEN views >= 350 THEN 1
          WHEN views < 100 THEN 3
          ELSE 2
       END AS group
FROM stackoverflow.users
WHERE location LIKE '%United States%' AND views > 0;
'''

### Задание 11
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
'''
WITH gr AS
(SELECT g.id,
       g.views,
       g.group,
       MAX(g.views) OVER (PARTITION BY g.group) AS max     
FROM (SELECT id,
             views,
             CASE
                 WHEN views >= 350 THEN 1
                 WHEN views < 100 THEN 3
                 ELSE 2
             END AS group
        FROM stackoverflow.users
        WHERE location LIKE '%United States%' AND views > 0) as g
)
SELECT gr.id,
       gr.group,
       gr.views
FROM gr
WHERE gr.views = gr.max
ORDER BY gr.views DESC, gr.id
'''

### Задание 12
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
'''
WITH day_c AS
(SELECT EXTRACT(DAY FROM creation_date) AS day_n,
       COUNT(id) AS count_id
FROM stackoverflow.users
WHERE CAST(DATE_TRUNC('MONTH', creation_date) AS DATE) = '2008-11-01'
GROUP BY EXTRACT(DAY FROM creation_date))

SELECT *,
       SUM(count_id) OVER (ORDER BY day_n)
FROM day_c
'''

### Задание 13
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:

- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
'''
   WITH f_post AS
(SELECT p.user_id,
       p.creation_date
FROM (SELECT user_id,
          creation_date,
          ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY creation_date) AS rn
   FROM stackoverflow.posts) AS p
WHERE p.rn = 1)

SELECT u.id,
       p.creation_date - u.creation_date
FROM stackoverflow.users AS u
JOIN f_post AS p ON u.id = p.user_id
'''



### Задание 14

Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. 

Результат отсортируйте по убыванию общего количества просмотров.


In [ ]:
'''
  SELECT CAST(DATE_TRUNC('month', creation_date) AS date),
       SUM(views_count)
FROM stackoverflow.posts
WHERE CAST(DATE_TRUNC('year', creation_date) AS DATE) = '2008-01-01'
GROUP BY CAST(DATE_TRUNC('month', creation_date) AS date)
ORDER BY SUM(views_count) DESC
'''

### Задание 15

Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. 

Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. 

Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
'''
  SELECT display_name,
       COUNT(DISTINCT p.user_id) AS total_answers
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON u.id=p.user_id
WHERE post_type_id=2
AND p.creation_date::date <= u.creation_date::date + INTERVAL '1 month'
GROUP BY display_name
HAVING COUNT(DISTINCT p.id) > 100
'''

### Задание 16
Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. 

Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
'''
  SELECT CAST(DATE_TRUNC('month', p.creation_date) AS date) AS month_posts,
         COUNT (DISTINCT id) AS user_count
    FROM stackoverflow.posts AS p 
   WHERE user_id IN (
                  SELECT DISTINCT u.id
                    FROM stackoverflow.users AS u
                    JOIN stackoverflow.posts AS p
                      ON p.user_id = u.id
                   WHERE u.creation_date :: date BETWEEN '2008-09-01' AND '2008-09-30'
                     AND p.creation_date :: date BETWEEN '2008-12-01' AND '2008-12-31'
                    )
     AND p.creation_date :: date BETWEEN '2008-01-01' AND '2008-12-31'
GROUP BY month_posts
ORDER BY month_posts DESC

'''

### Задание 17
Используя данные о постах, выведите несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
'''
  SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY user_id
'''

### Задание 18
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? 

Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
'''
WITH ad AS (SELECT user_id,
                   COUNT(DISTINCT creation_date::date)  AS cnt
            FROM stackoverflow.posts
            WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07' 
            GROUP BY user_id)

SELECT ROUND(AVG(ad.cnt))
FROM ad;

'''

### Задание 19
На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.


In [ ]:
'''
WITH t AS
(SELECT EXTRACT('MONTH' FROM creation_date) AS month_numb,
       COUNT(id) AS count_id,
       COUNT(id)::numeric - LAG(COUNT(id), 1, COUNT(id)) OVER (ORDER BY EXTRACT('MONTH' FROM creation_date)) AS cht
FROM stackoverflow.posts
WHERE CAST(creation_date AS DATE) BETWEEN '2008-09-01' AND '2008-12-31'
GROUP BY EXTRACT('MONTH' FROM creation_date))

SELECT month_numb,
       count_id,
       ROUND(((cht * 100) / LAG(count_id) OVER (ORDER BY month_numb))::numeric, 2)
FROM t
'''

### Задание 20

Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
'''
WITH
user_numb AS
(SELECT user_id,
       COUNT(DISTINCT id)
FROM stackoverflow.posts
GROUP BY user_id
ORDER BY COUNT(id) DESC
LIMIT 1),
post_inf AS
(SELECT p.id,
       p.creation_date,
       EXTRACT('WEEK' FROM p.creation_date) AS week
FROM stackoverflow.posts AS p
JOIN user_numb AS un ON p.user_id = un.user_id
WHERE DATE_TRUNC('month', p.creation_date) = '2008-10-01')

SELECT DISTINCT week::numeric,
       MAX(creation_date) OVER (PARTITION BY week) 
FROM post_inf
'''   